In [1]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [2]:
cd /content/drive/MyDrive/inter_iit

/content/drive/MyDrive/inter_iit


In [3]:
!pip install mahotas

In [4]:
pip install numpy --upgrade

In [5]:
import os
import cv2
import numpy as np
import pandas as pd
import mahotas as mt
from matplotlib import pyplot as plt
%matplotlib inline

In [6]:
dest = r'/content/drive/MyDrive/inter_iit/Leaves/'
images = os.listdir(dest)

In [7]:
category_points = [1001,1059,1060,1122,1552,1616,1123,1194,1195,1267,1268,1323,1324,1385,1386,1437,1497,1551,1438,1496,2001,2050,2051,2113,2114,2165,2166,2230,2231,2290,2291,2346,2347,2423,2424,2485,2486,2546,2547,2612,2616,2675,3001,3055,3056,3110,3111,3175,3176,3229,3230,3281,3282,3334,3335,3389,3390,3446,3447,3510,3511,3563,3566,3621]

In [11]:
def dataset_creation():
    attributes = ['area','perimeter','length','width','aspect_ratio','rectangularity','circularity', 'mean_r','mean_g','mean_b','stdd_r','stdd_g','std_b', 'contrast','correlation','inverse_difference_moments','entropy','target']
    dataframe = pd.DataFrame([], columns=attributes)
    for imgs in images:
        img_path = dest + imgs
        main_image = cv2.imread(img_path)
        
        image = cv2.cvtColor(main_image, cv2.COLOR_BGR2RGB)
        gray = cv2.cvtColor(image,cv2.COLOR_RGB2GRAY)
        blur = cv2.GaussianBlur(gray, (25,25),0)
        ret_otsu,im_bw_otsu = cv2.threshold(blur,0,255,cv2.THRESH_BINARY_INV+cv2.THRESH_OTSU)
        kernel = np.ones((50,50),np.uint8)
        closing = cv2.morphologyEx(im_bw_otsu, cv2.MORPH_CLOSE, kernel)
        
        contours, _ = cv2.findContours(closing,cv2.RETR_TREE,cv2.CHAIN_APPROX_SIMPLE)
        cnt = contours[0]
        M = cv2.moments(cnt)
        area = cv2.contourArea(cnt)
        perimeter = cv2.arcLength(cnt,True)
        p,q,r,s = cv2.boundingRect(cnt)
        aspect_ratio = float(r)/s
        rectangularity = r*s/area
        circularity = ((perimeter)**2)/area
        
        red = image[:,:,0]
        green = image[:,:,1]
        blue = image[:,:,2]
        blue[blue == 255] = 0
        green[green == 255] = 0
        red[red == 255] = 0
        
        mean_r = np.mean(red)
        mean_g = np.mean(green)
        mean_b = np.mean(blue)
        
        stdd_r= np.std(red)
        stdd_g = np.std(green)
        stdd_b = np.std(blue)
        
        texture = mt.features.haralick(gray)
        ht_mean = texture.mean(axis=0)
        contrast = ht_mean[1]
        correlation = ht_mean[2]
        inverse_diff_moments = ht_mean[4]
        entropy = ht_mean[8]
        target=''
        image_number = int(imgs.split(".")[0])
        f = 0
        i = 0 
        for i in range(0,len(category_points),2):
          if((image_number >= category_points[i]) and (image_number <= category_points[i+1])):
            f = 1
            break
        if(f==1):
         target = int((i/2))
         

        vector = [area,perimeter,r,s,aspect_ratio,rectangularity,circularity,mean_r,mean_g,mean_b,stdd_r,stdd_g,stdd_b,contrast,correlation,inverse_diff_moments,entropy,target]
        
        temp = pd.DataFrame([vector],columns=attributes)
        dataframe = dataframe.append(temp)
    return dataframe

In [12]:
dataset = dataset_creation()

In [13]:
dataset.to_csv("leaves.csv")